In [ ]:
import pandas as pd
import numpy as np

In [ ]:
import indicators
import myutils
import sys

In [ ]:
def MACD_Histogram(data, short_window, long_window, signal_window):
    try:
        
        if (len(data) < (long_window + signal_window - 1)):
            print(sys._getframe().f_code.co_name,'Insufficient data')
            return None
        
        if (short_window < 1):
            print(sys._getframe().f_code.co_name,'Invalid parameter value for short_window')
            return None
        
        if (long_window <= short_window):
            print(sys._getframe().f_code.co_name,'Invalid parameter value for long_window')
            return None
        
        if (signal_window < 1):
            print(sys._getframe().f_code.co_name,'Invalid parameter value for signal_window')
            return None
        
        data = pd.DataFrame(data[-(long_window + signal_window -1):])
        
        data['Short_MA'] = data['Close'].ewm(span = short_window, adjust = False).mean()
        
        data['Long_MA'] = data['Close'].ewm(span = long_window, adjust = False).mean()
        
        data['MACD'] = data['Short_MA'] - data['Long_MA']
        
        data['MACD_Signal'] = data['MACD'].ewm(span = signal_window, adjust = False).mean()
        
        data['MACD_Histogram'] = data['MACD'] - data['MACD_Signal']
        
        MACD_Histogram = float(data['MACD_Histogram'].iloc[-1])
        
        return MACD_Histogram
        
    except Exception as ex:
        print(sys._getframe().f_code.co_name, 'exception :', ex)
        return None


In [ ]:
def generate_signal_MACD_Histogram(symbol_name, ltp, short_window, long_window, signal_window, long_entry, short_entry, data_path ):
    try:
        
        symbol_name = symbol_name.strip()
        
        if type(ltp) == str:
            ltp = float(ltp.strip())
        
        if type(short_window) is str:
            short_window = int(short_window.strip())
       
        if type(long_window) is str:
            long_window = int(long_window.strip())
       
        if type(signal_window) is str:
            signal_window = int(signal_window.strip())
        
        if type(long_entry) is str:
            long_entry = float(long_entry.strip())
        
        if type(short_entry) is str:
            short_entry = float(short_entry.strip())
        
        if (symbol_name == '' or ltp <= 0 or short_window < 1 or long_window <= short_window or signal_window < 1 or long_entry < 1 or short_entry <= 0 or short_entry > 1):
            print(sys._getframe().f_code.co_name,'Invalid parameter values')
            return None
        
        data = myutils.read_dataframe(data_path + '/' + symbol_name + '.csv')
        
        if (data.empty):
            print(sys._getframe().f_code.co_name,'Data not found')
            return None
        
        # specific to your data set, modify accordingly
        data = data['Close']
        
        data = data.dropna()
        
        #data = data[-period:]
        
        current_macd_histogram = indicators.MACD_Histogram(data, short_window, long_window, signal_window)
        
        if (current_macd_histogram == None):
            return None
        
        print('MACD Histogram ' + str(current_macd_histogram))
        
#        data['Long_MA'] = data['Close'][-long_window:].ewm(span = long_window, adjust = False).mean()
        
#        LMA = data['Long_MA'][-1]
        
        if (current_macd_histogram > 0):
            signal = 1
        elif (current_macd_histogram < 0):
            signal = -1
        else:
            signal = 0
        
        return signal
        
    except Exception as ex:
        print(sys._getframe().f_code.co_name, 'exception :', ex)
        return None
            